In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [3]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [4]:
from collections import defaultdict

In [5]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [6]:
len(hashes), len(documents)

(947, 948)

In [7]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [8]:
import json

In [9]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [9]:
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


In [10]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [11]:
from openai import OpenAI
client = OpenAI()

In [12]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [13]:
from tqdm.auto import tqdm

In [14]:
results = {}

In [15]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|          | 0/948 [00:00<?, ?it/s]

In [18]:
results['1f6520ca']

'[\n  "What GitHub link contains the prerequisites for the course?",\n  "Where can I find the prerequisites for the data engineering zoomcamp?",\n  "Can you provide the GitHub link to check the course prerequisites?",\n  "Which GitHub repository details the prerequisites for this course?",\n  "Is there a specific GitHub page for the prerequisites of the data engineering zoomcamp?"\n]'

In [64]:
parsed_resulst = {}

for doc_id, json_questions in results.items():
    parsed_resulst[doc_id] = json.loads(json_questions)

In [65]:
doc_index = {d['id']: d for d in documents}

In [66]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [67]:
import pandas as pd

In [68]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [69]:
df.to_csv('ground-truth-data.csv', index=False)

In [70]:
!head ground-truth-data.csv

question,course,document
When will the course start?,data-engineering-zoomcamp,c02e79ef
What time will the course begin on its first day?,data-engineering-zoomcamp,c02e79ef
When should I register for the course?,data-engineering-zoomcamp,c02e79ef
Where can I see course announcements?,data-engineering-zoomcamp,c02e79ef
How can I join the course Telegram channel?,data-engineering-zoomcamp,c02e79ef
What GitHub link contains the prerequisites for the course?,data-engineering-zoomcamp,1f6520ca
Where can I find the prerequisites for the data engineering zoomcamp?,data-engineering-zoomcamp,1f6520ca
Can you provide the GitHub link to check the course prerequisites?,data-engineering-zoomcamp,1f6520ca
Which GitHub repository details the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
